In [1]:
! pip install pandas

In [2]:
!pip install numpy

In [3]:
import pandas as pd
import numpy as np

In [4]:
news_data = pd.read_csv("C:\\Users\\anjup\\Downloads\\abcnews-date-text.csv")
news_data.shape


(1244184, 2)

In [5]:
NUM_SAMPLES = 20000 # The number of sample to use

df = news_data.sample(NUM_SAMPLES, replace=False, random_state=42).reset_index(drop=True)

In [6]:
df.head()

,publish_date,headline_text
0,20181017,virtual reality trial ahead of fire season in ...
1,20070131,farmers prepare for ec funding
2,20140810,the sunday inquisition august 10
3,20130221,news csg reax
4,20140806,rosetta spacecraft on final approach to comet ...


In [7]:
from transformers import AutoModel, AutoTokenizer
model = AutoModel.from_pretrained("google/muril-base-cased")
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")


In [8]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from sentence_transformers import SentenceTransformer
import hdbscan
from gensim.models import CoherenceModel
from gensim import corpora
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import defaultdict
import nltk

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# --- Step 1: Preprocess text ---
stop_words = set(stopwords.words('english'))

def preprocess(doc):
    tokens = word_tokenize(doc.lower())
    return [t for t in tokens if t.isalpha() and t not in stop_words]
    # Load and preprocess
docs_raw = df['headline_text'].dropna().tolist()
texts = [preprocess(doc) for doc in docs_raw]
docs = [" ".join(t) for t in texts]

# --- Step 2: Generate BERT or MuRIL Embeddings ---
# You can swap the model to 'ai4bharat/indic-sentence-bert' for MuRIL
model = model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

embeddings = model.encode(docs, show_progress_bar=True)

# --- Step 3: HDBSCAN Clustering ---
clusterer = hdbscan.HDBSCAN(min_cluster_size=15, metric='euclidean')
labels = clusterer.fit_predict(embeddings)

# --- Step 4: TF-IDF for topic word extraction ---
valid_idx = labels != -1
filtered_docs = np.array(docs)[valid_idx]
filtered_labels = labels[valid_idx]

vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(filtered_docs)
terms = vectorizer.get_feature_names_out()

cluster_docs = defaultdict(list)
for doc, label in zip(filtered_docs, filtered_labels):
    cluster_docs[label].append(doc)
    topic_words = []
for label, cluster_texts in cluster_docs.items():
    tfidf_cluster = vectorizer.fit_transform(cluster_texts)
    mean_tfidf = np.asarray(tfidf_cluster.mean(axis=0)).flatten()
    top_n_idx = mean_tfidf.argsort()[-10:][::-1]
    topic = [terms[i] for i in top_n_idx]
    topic_words.append(topic)

# --- Step 5: Coherence Score ---
dictionary = corpora.Dictionary(texts)
coherence_model = CoherenceModel(
    topics=topic_words,
    texts=texts,
    dictionary=dictionary,
    coherence='c_v',
    topn=10
)
coherence_score = coherence_model.get_coherence()

# --- Step 6: Silhouette Score ---
if np.unique(filtered_labels).size > 1:
    sil_score = silhouette_score(embeddings[valid_idx], filtered_labels)
else:
    sil_score = None

# --- Step 7: Output Results ---
print("Coherence Score (MuRIL Hybrid):", round(coherence_score, 4))
print("Silhouette Score (MuRIL Hybrid):", round(sil_score, 4) if sil_score else "Not enough distinct clusters")
    

ModuleNotFoundError: No module named 'hdbscan'

In [ ]:
unique_clusters = sorted(set(labels[labels != -1]))  # exclude noise (-1)

user_type_map = {cluster: f"UserType_{i}" for i, cluster in enumerate(unique_clusters)}

# Map cluster labels to user types
user_type_predictions = [user_type_map[label] if label in user_type_map else "Unknown" for label in labels]

# Add to DataFrame
df = df.loc[df['headline_text'].notna()].copy()  # Ensure alignment
df['predicted_user_type'] = user_type_predictions

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['topic_label'] = label_encoder.fit_transform(df['predicted_user_type'])

# Final labels
y = df['topic_label'].values

In [ ]:
# Keep only non-noise docs
X = embeddings[labels != -1]
y = y[labels != -1]  # ensure same indexing

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50, random_state=42)
X_reduced = pca.fit_transform(X)  # Before train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
log_reg = LogisticRegression(C=0.1, penalty='l2', max_iter=1000)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(log_reg, X_reduced, y, cv=5)
print("Cross-validated accuracy (Logistic):", scores.mean())

Cross-validated accuracy (Logistic): 0.9979591836734695
